In [ ]:
import pandas as pd
import os
# import lxml
import DB as db

In [ ]:
projects= (66, 67, 68, 69)
# files = ('Results-66.txt', 'Results-67.txt', 'Results-68.txt', 'Results-69.txt')
dir = r"C:\Users\yahia\OneDrive - Data and Transaction Services\Projects\Realestate Reservation Portal\5- Operation\Incident reports\change land-no\checksum"

lst = []
for p in projects:

    fpath = os.path.join(dir, f"Results-{p}.txt") 
    with open(fpath, 'rt') as f:
        while(True):
            line = f.readline()
            if not line: break
            if line[:5] in ('-----', '     '): continue
            NID = line[25:39]
            land_id = line[51:56]
            res_no = line[79:93]
            valid = line[98:106].strip()
            rec = [p, NID, land_id, res_no, valid]
            lst.append(rec)

df = pd.DataFrame(data=lst, columns=['project', 'NID', 'land_id', 'reservation_no', 'valid'])
df.to_csv('./out/checksum.csv')

In [ ]:
df1=pd.DataFrame()
df1['NID'] = df.iloc[:,0].str[25:41]
df1['land_id'] = df.iloc[:,1].str[10:16]
df1['res_no'] = df.iloc[:,2].str[22:35]
df1['valid'] = df.iloc[:,2].str[40:50]
df1.loc[df1.NID.str.startswith('---')]
# df1

In [ ]:
dblog_csv_fn = r"C:\Yahia\Python\portal_logs\out\dblog_from-to_db_log.csv"
# df = pd.read_csv(dblog_csv_fn, parse_dates=True, usecols=['tmstamp', 'query','params','proc_tbl'])
df = pd.read_csv(dblog_csv_fn, parse_dates=True, usecols=['params','proc_tbl'])
proc_name = 'reserveLand'
tbl = df.loc[df.proc_tbl == proc_name]    

In [ ]:
t = tbl.params.str.split(',')
# t.apply(lambda x: x[2])
t.apply(lambda x: len(x))


In [ ]:
nparams = len(tbl.iloc[0,1].split(','))
df = pd.DataFrame()
t = tbl.params.str.split(',')
for i in range(nparams) :
    df[f"P{i+1}"] = t.apply(lambda x: x[i])

df

In [ ]:
{"Gov":"القاهرة","City":"القاهرة الجديدة","Region":"نزهة التجمع الثالث بمنطقة المستقبل (3)","District":"null","Sub_District":"null","Floor_No":"0","building_no":"2","Unit_No":"15","Unit_Model":"A","Unit_ID":"118744","checksum":"01AE4AA44C9C78B81EFC0443D02AC43C687F023ECCB8A07631FEC86258B50CE7F6"}

In [ ]:
import tarfile
fn = r"C:\Users\yahia\Downloads\server.log.2022-12-28.tar.gz"

file_name = None
file_obj = fn
with tarfile.open(name  = fn, fileobj = file_obj, mode = "r:gz") as tar:
    for member in tar.getmembers():
        f=tar.extractfile(member)
        for f in zfiles:
        line_no = 0
        # print (fname)
        while True:
            txt = f.readline()
            # print (len(txt), txt)
            if not txt: break # end of file
            line_no += 1

            # if line_no > 10000: break
            # if line_no < 100000000: continue
            # if line_no % 10000 == 0: print (line_no)
            if line_no % 40000 == 0: print (line_no)
            
            txt = txt.decode('utf-8')
            log_type = txt[24:29]